In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

def create_environment(cities = ['A','B','C','D','E','F','G'],  path = ['A', 'F', 'D', 'E']):
    def distance_matrix(cities, path):
        N =len(cities)
        env = pd.DataFrame(data=np.ones((N,N)) * 10, columns=cities, index = cities)
        
        ## -1: disallowed passage
        for i in range(N):
            env.loc[cities[i], cities[i]] = -1

        for i in range(len(path)-1):
            env.loc[path[i], path[i+1]] = 1
            env.loc[path[i+1], path[i]] = 1

        return env

    def pheremon_matrix(cities, eps = 0.0001):
        N =len(cities)
        return pd.DataFrame(data=np.ones((N,N)) * eps, columns=cities, index = cities)

    return distance_matrix(cities, path), pheremon_matrix(cities)

In [2]:
env, phe = create_environment()
env

,A,B,C,D,E,F,G
A,-1.0,10.0,10.0,10.0,10.0,1.0,10.0
B,10.0,-1.0,10.0,10.0,10.0,10.0,10.0
C,10.0,10.0,-1.0,10.0,10.0,10.0,10.0
D,10.0,10.0,10.0,-1.0,1.0,1.0,10.0
E,10.0,10.0,10.0,1.0,-1.0,10.0,10.0
F,1.0,10.0,10.0,1.0,10.0,-1.0,10.0
G,10.0,10.0,10.0,10.0,10.0,10.0,-1.0


In [3]:
phe

,A,B,C,D,E,F,G
A,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
B,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
C,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
D,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
E,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
F,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
G,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [4]:
class ant():
    def __init__(self, env = env, phe = phe, 
                 start = 'A', end = 'E',
                alpha = 1, beta = 1):
        
        self.env, self.phe  = env, phe
        self.alpha, self.beta = alpha, beta
        self.cities = list(self.env.columns)
        self.current_city = start
        self.target_city = end
        
        self.route = [self.current_city]
        
        self.possible_cities = self.cities.copy()
        self.possible_cities.remove(self.current_city)
    
    def move(self):
        distances = env.loc[self.current_city , env.loc[self.current_city ] > 0][self.possible_cities]
        pheremons = phe.loc[self.current_city , env.loc[self.current_city ] > 0][self.possible_cities]
        
        preferences = pheremons**self.alpha/distances**self.beta
        probabilities = preferences/preferences.sum()
        # print(probabilities)
        
        self.current_city = np.random.choice(a = probabilities.index, 
                                             size=1, 
                                             p = probabilities.values)[0]
        
        self.route.append(self.current_city) 
        self.possible_cities.remove(self.current_city)
        
        self.deposit()

    def deposit(self, delta = 0.01):
        i, j = self.route[-2:]
        self.phe.loc[i, j] +=  delta
        self.phe.loc[j, i] +=  delta
        
    def run(self):
        while self.current_city != self.target_city:
            self.move()
         
        result = 0
        for i in range(len(self.route)-1):
            result += self.env.loc[self.route[i], self.route[i+1]]
        return result

In [5]:
for i in range(50):
    a = ant()
    print(a.run(), a.route)
    phe = phe * 0.9

22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
3.0 ['A', 'F', 'D', 'E']
42.0 ['A', 'F', 'D', 'C', 'B', 'G', 'E']
3.0 ['A', 'F', 'D', 'E']
31.0 ['A', 'F', 'C', 'B', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'D', 'C', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A', 'F', 'C', 'D', 'E']
22.0 ['A',

In [6]:
phe

,A,B,C,D,E,F,G
A,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.205313e-05,5.153775e-07
B,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07
C,5.153775e-07,5.153775e-07,5.153775e-07,5.205313e-05,5.153775e-07,5.205313e-05,5.153775e-07
D,5.153775e-07,5.153775e-07,5.205313e-05,5.153775e-07,5.205313e-05,5.153775e-07,5.153775e-07
E,5.153775e-07,5.153775e-07,5.153775e-07,5.205313e-05,5.153775e-07,5.153775e-07,5.153775e-07
F,5.205313e-05,5.153775e-07,5.205313e-05,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07
G,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07,5.153775e-07
